In [2]:
import os
import csv
import pandas as pd
import numpy as np
import math
import time

from sklearn.linear_model import Ridge
import xgboost as xgb

In [3]:
path_train = "train.csv"  # 训练文件
path_test = "test.csv"  # 测试文件

path_test_out = "model/"  # 预测结果输出路径为model/xx.csv,有且只能有一个文件并且是CSV格式。

In [4]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt

def conver_time(data):
    data['Conver_TIME'] = data.TIME.apply(timestamp_datetime)
#     data['month'] = data.Conver_TIME.apply(lambda x: int(x[5:7]))
    data['hour'] = data.Conver_TIME.apply(lambda x: int(x[11:13]))
#     data = data.drop('TIME',axis=1)
#     data = data.drop('Conver_TIME',axis=1)
    return data

def label_process(data):
    pre_label = data.drop_duplicates()
    return pre_label['Y'].values

def speed_map(speed):
    if speed <= 4.5:
        return 1
    elif (speed > 4.5) and (speed <= 15):
        return 2
    elif (speed > 15) and (speed <= 31.5):
        return 3
    else:
        return 4


def height_map(hegiht):
    if hegiht <= 91.5:
        return 1
    elif (hegiht > 91.5) and (hegiht <= 451):
        return 2
    elif (hegiht > 451) and (hegiht <= 1570):
        return 3
    else:
        return 4


def speed_height(speed, height):
    if height <= 451:
        return 1
    elif height > 451 and height <= 1570 and speed > 15:
        return 2
    elif height > 451 and height <= 1570 and speed > 31.5:
        return 3
    elif height > 1570 and speed > 31.5:
        return 4
    else:
        return 5


def longtitude_map(longti):
    if (longti >= 118) and (longti <= 123):
        return 1
    elif (longti >= 105) and (longti <= 118):
        return 2
    elif (longti >= 101) and (longti < 105) or (longti > 123) and (longti <= 127):
        return 3
    else:
        return 4


def direction_map(direction):
    if direction >= 0 and direction < 90:
        return 1
    elif direction >= 90 and direction < 180:
        return 2
    elif direction >= 180 and direction < 270:
        return 3
    elif direction >= 270 and direction < 360:
        return 4
    else:
        return 5


def speed_direction(speed, direction):
    if direction < 180 and speed < 31.5:
        return 1
    elif direction < 180 and speed > 31.5:
        return 2
    elif direction > 180 and direction < 360 and speed < 31.5:
        return 3
    elif direction > 180 and direction < 360 and speed > 31.5:
        return 4
    else:
        return 5


def time_convert(timestamp, type):
    #转换成localtime
    time_local = time.localtime(timestamp)
    if type == 'hour':
        #转换成新的时间格式(2016-05-05 20:28:54)
        # dt = time.strftime("%Y-%m-%d %H:%M:%S", time_local)
        dt = time.strftime("%H", time_local)
    else:
        dt = time.strftime("%w", time_local)
    return dt


# 加载训练集
# def load_trainData(trainFilePath):
#     df = pd.read_csv(trainFilePath)
#     data = df.copy()
#     data['DIRECTION'] = data['DIRECTION'].apply(direction_map)
#     data['SPEED'] = data['SPEED'].apply(speed_map)
#     data['HEIGHT'] = data['HEIGHT'].apply(height_map)
#     data['LONGITUDE'] = data['LONGITUDE'].apply(longtitude_map)
#     data['SPEED_HEIGHT'] = data.apply(
#         lambda col: speed_height(col['SPEED'], col['HEIGHT']), axis=1)
#     data['SPEED_DIRECTION'] = data.apply(
#         lambda col: speed_direction(col['SPEED'], col['DIRECTION']), axis=1)
#     data['Hour'] = data['TIME'].apply(lambda x: time_convert(x, 'hour'))
#     data['Week'] = data['TIME'].apply(lambda x: time_convert(x, 'week'))
#     data[['Hour', 'Week']] = data[['Hour', 'Week']].apply(pd.to_numeric)
#     data['isNight'] = data['Hour'].apply(
#         lambda x: 0 if (x > 5 and x < 19) else 1)
#     data['isWeekend'] = data['Week'].apply(
#         lambda x: 0 if (x > 0 and x < 6) else 1)
#     data.drop(
#         ['TERMINALNO', 'TIME', 'TRIP_ID', 'Hour', 'Week', 'Y'],
#         axis=1,
#         inplace=True)
#     return data, df['Y']


# 加载测试集
# def load_testData(testFilePath):
#     df = pd.read_csv(testFilePath)
#     data = df.copy()
#     data['DIRECTION'] = data['DIRECTION'].apply(direction_map)
#     data['SPEED'] = data['SPEED'].apply(speed_map)
#     data['HEIGHT'] = data['HEIGHT'].apply(height_map)
#     data['LONGITUDE'] = data['LONGITUDE'].apply(longtitude_map)
#     data['SPEED_HEIGHT'] = data.apply(
#         lambda col: speed_height(col['SPEED'], col['HEIGHT']), axis=1)
#     data['SPEED_DIRECTION'] = data.apply(
#         lambda col: speed_direction(col['SPEED'], col['DIRECTION']), axis=1)
#     data['Hour'] = data['TIME'].apply(lambda x: time_convert(x, 'hour'))
#     data['Week'] = data['TIME'].apply(lambda x: time_convert(x, 'week'))
#     data[['Hour', 'Week']] = data[['Hour', 'Week']].apply(pd.to_numeric)
#     data['isNight'] = data['Hour'].apply(
#         lambda x: 0 if (x > 5 and x < 19) else 1)
#     data['isWeekend'] = data['Week'].apply(
#         lambda x: 0 if (x > 0 and x < 6) else 1)
#     data.drop(
#         ['TERMINALNO', 'TIME', 'TRIP_ID', 'Hour', 'Week'],
#         axis=1,
#         inplace=True)
#     return data, df['TERMINALNO']

def raw_process(data):
    data['DIRECTION'] = data['DIRECTION'].apply(direction_map)
    data['SPEED'] = data['SPEED'].apply(speed_map)
    data['HEIGHT'] = data['HEIGHT'].apply(height_map)
    data['LONGITUDE'] = data['LONGITUDE'].apply(longtitude_map)
    data['SPEED_HEIGHT'] = data.apply(
        lambda col: speed_height(col['SPEED'], col['HEIGHT']), axis=1)
    data['SPEED_DIRECTION'] = data.apply(
        lambda col: speed_direction(col['SPEED'], col['DIRECTION']), axis=1)
    data['Week'] = data['TIME'].apply(lambda x: time_convert(x, 'week'))
    data['Week'] = data['Week'].apply(pd.to_numeric)
    data['isWeekend'] = data['Week'].apply(
        lambda x: 0 if (x > 0 and x < 6) else 1)
    data.drop(
        ['TERMINALNO', 'TIME', 'TRIP_ID', 'Week'],
        axis=1,
        inplace=True)
    return data
    

def feature_process(data):
    data = conver_time(data)
    set_data = set(data['TERMINALNO'])
    columns=['id','maxTime', 'phonerisk', 'dir_risk', 'height_risk', 'speed_max', 'speed_mean', 'height_mean','zao','wan','shenye']
    feature = pd.DataFrame(columns=columns)
    for p_id in set_data:
        tempData = data.loc[data['TERMINALNO'] == p_id]      
        tempData = tempData.sort_values(["TIME"])
    
        tempTime = tempData["TIME"].iloc[0]
        tempSpeed = tempData["SPEED"].iloc[0]
        tempDir = tempData["DIRECTION"].iloc[0]
        tempHeight = tempData["HEIGHT"].iloc[0]
        
        # 根据时间信息判断最长时间
        maxTime = 0
        maxTimelist = []

        # 用户行驶过程中，打电话危机上升
        phonerisk = 0

        # Direction 突变超过
        dir_risk = 0

        # Height 高度的危险值
        height_risk = 0
        zao=0
        # wu=0
        wan=0
        shenye=0
        for index, row in tempData.iterrows():
            hour = row['hour']
            # if hour>=3 and hour <=5:
            #     zao=1
            # elif hour>=6 and hour<=8:
            #     wu=1
            # elif hour>=9 and hour<=11:
            #     wan=1
            # else:
            #     shenye=1
            if 7 <= hour <= 9:
                zao = 1
            elif 17 <= hour <= 19:
                wan = 1
            elif 0 <= hour < 7:
                shenye = 1

            if tempSpeed > 0 and row['CALLSTATE'] != 4:
                if row["CALLSTATE"] == 0:
                    phonerisk += math.exp(tempSpeed / 10) * 0.02
                else:
                    phonerisk += math.exp(tempSpeed / 10)

        
            if row["TIME"] - tempTime == 60:
                maxTime += 60
                tempTime = row["TIME"]

                # 判断方向变化程度与具有车速之间的危险系数
                dir_change = (min(abs(row["DIRECTION"] - tempDir), abs(360 + tempDir - row["DIRECTION"])) / 90.0)
                if tempSpeed != 0 and row["SPEED"] > 0:
                    dir_risk += math.pow((row["SPEED"] / 10), dir_change)
                
                # 海拔变化大的情况下和速度的危险系数
                height_risk += math.pow(abs(row["SPEED"] - tempSpeed) / 10,(abs(row["HEIGHT"] - tempHeight) / 100))
                
                tempHeight = row["HEIGHT"]

            elif row["TIME"] - tempTime > 60:
                maxTimelist.append(maxTime)
                maxTime = 0
                tempTime = row["TIME"]

                tempDir = row["DIRECTION"]
                tempHeight = row["HEIGHT"]
                tempSpeed = row["SPEED"]
                
        speed_max = tempData["SPEED"].max()
        speed_mean = tempData["SPEED"].mean()

        height_mean = tempData["HEIGHT"].mean()

        maxTimelist.append(maxTime)
        maxTime = max(maxTimelist)
        
        tempfeature = pd.DataFrame({'id':p_id,
                                    'maxTime':maxTime,
                                    'phonerisk':phonerisk, 
                                    'dir_risk':dir_risk, 
                                    'height_risk':height_risk, 
                                    'speed_max':speed_max, 
                                    'speed_mean':speed_mean, 
                                    'height_mean':height_mean,
                                    'zao':zao,
                                    'wan':wan,
                                    'shenye':shenye},index=['0'],columns=columns)
        feature = feature.append(tempfeature,ignore_index=True)

#     feature = feature.values
    return feature

In [5]:
X_train, Y_train = load_trainData(path_train)
X_test, IdList = load_testData(path_test)

In [6]:
X_train.head()

,LONGITUDE,LATITUDE,DIRECTION,HEIGHT,SPEED,CALLSTATE,SPEED_HEIGHT,SPEED_DIRECTION,isNight,isWeekend
0,1,41.103741,1,1,1,0,1,1,0,0
1,1,41.104904,1,1,1,0,1,1,0,0
2,1,41.106388,1,1,1,0,1,1,0,0
3,1,41.106884,2,1,2,0,1,1,0,0
4,1,41.105442,2,1,1,0,1,1,0,0


In [8]:
X_train.shape

(69306, 10)

In [7]:
train = pd.read_csv(path_train)
test = pd.read_csv(path_test)
# pre_label = label_process(train[["TERMINALNO","Y"]])
train = train.drop('Y',axis=1)

In [8]:
feature_train = feature_process(train)
feature_test = feature_process(test)

In [9]:
feature_train.head()

,id,maxTime,phonerisk,dir_risk,height_risk,speed_max,speed_mean,height_mean,zao,wan,shenye
0,1.0,3540.0,22.698271,759.505736,253.833059,32.779999,17.489840,47.848093,1.0,1.0,1.0
1,2.0,3180.0,67.556593,3646.161361,376.753980,36.119999,9.287734,71.400718,0.0,1.0,1.0
2,3.0,3600.0,41.291745,597.444549,840.111283,25.440001,7.987331,43.942672,1.0,1.0,0.0
3,4.0,2700.0,110.310492,659.554444,656.575946,33.310001,6.312753,31.659250,1.0,1.0,0.0
4,5.0,5520.0,39.709307,616.796195,776.837669,53.480000,7.695846,29.461915,1.0,1.0,0.0
